In [48]:
import qiskit
import numpy as np

from qiskit_aer import AerSimulator
from qiskit.providers import fake_provider
from qiskit import QuantumCircuit
from mitiq import zne, ddd
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise

In [49]:
a, b = 0, 1  
circuit = QuantumCircuit(2)
circuit.h(a)
circuit.h(b)
circuit.rz(1.75, a)
circuit.rz(2.31, b)
circuit.cx(a, b)
circuit.rz(-1.17, b)
circuit.rz(3.23, a)
circuit.rx(np.pi / 2, a)
circuit.rx(np.pi / 2, b)
circuit.measure_all()
print(circuit)

backend_name = getattr(fake_provider, "FakeKolkata")()
backend = AerSimulator.from_backend(backend_name)

noise_level=0.01
number_of_shots = 8192

def executor(circuit: QuantumCircuit, noise_level = noise_level) -> float:
    noise_model = initialized_depolarizing_noise(noise_level=noise_level)
    job = qiskit.execute(
        experiments=circuit,
        backend=backend,
        noise_model=noise_model,
        basis_gates=noise_model.basis_gates,
        optimization_level=0,# Important to preserve folded gates.
        shots=number_of_shots,
    )
    counts = job.result().get_counts()

    zero_prob = counts.get("0"*circuit.num_qubits, 0)/number_of_shots
    
    return zero_prob
    # expectation_value = 0
    # for measurement, count in counts.items():
    #     parity = sum(int(bit) for bit in measurement) % 2 
    #     if parity == 0:
    #         expectation_value += count /  number_of_shots
    #     else:
    #         expectation_value -= count / number_of_shots
    
    # return expectation_value

noisy_value = executor(circuit)
ideal_value = executor(circuit, noise_level=0.0)

        ┌───┐┌──────────┐      ┌──────────┐┌─────────┐ ░ ┌─┐   
   q_0: ┤ H ├┤ Rz(1.75) ├──■───┤ Rz(3.23) ├┤ Rx(π/2) ├─░─┤M├───
        ├───┤├──────────┤┌─┴─┐┌┴──────────┤├─────────┤ ░ └╥┘┌─┐
   q_1: ┤ H ├┤ Rz(2.31) ├┤ X ├┤ Rz(-1.17) ├┤ Rx(π/2) ├─░──╫─┤M├
        └───┘└──────────┘└───┘└───────────┘└─────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════════════════════════════╩══╩═
                                                          0  1 


In [50]:
mitigated = zne.execute_with_zne(circuit, executor)

print(f"Error without mitigation: {abs(ideal_value - noisy_value) :.5f}")
print(f"Error with mitigation (ZNE): {abs(ideal_value - mitigated):.5f}")

Error without mitigation: 0.02271
Error with mitigation (ZNE): 0.03271


In [51]:
#custom flow that allows transpile circuit, generate noise scaled circuits, run DDD on each of them and later extrapolate
transpiled_circuit = qiskit.transpile(circuit)

scale_factors = [1.0, 3.0, 5.0]
noise_scaled_circuits = [zne.scaling.fold_gates_at_random(transpiled_circuit, s) for s in scale_factors]

#start ddd
rule = ddd.rules.yy
ddd_mitigated_values = [ddd.execute_with_ddd(circuit=c, executor=executor, rule=rule) for c in noise_scaled_circuits]
#end ddd

#assuming an infinite noise limit of 0.5
fac = zne.inference.ExpFactory(scale_factors, asymptote=0.5)
for s, e in zip(scale_factors, ddd_mitigated_values):
    fac.push({"scale_factor": s}, e)
reduced_result = fac.reduce()

#results check
print(f"Error without mitigation-----------------------------------: {abs(ideal_value - noisy_value) :.5f}")
print(f"Error with mitigation (transpile + ZNE (DDD in the middle)): {abs(ideal_value - reduced_result):.5f}")



Error without mitigation-----------------------------------: 0.02271
Error with mitigation (transpile + ZNE (DDD in the middle)): 0.00471
